In [177]:
import pandas as pd
import numpy as np
import opendatasets as od
import time
import datetime
import torch.nn as nn

import torch
from sklearn.model_selection import train_test_split

In [165]:
dataset_url = 'https://www.kaggle.com/competitions/new-york-city-taxi-fare-prediction'
od.download(dataset_url)

Skipping, found downloaded files in "./new-york-city-taxi-fare-prediction" (use force=True to force download)


In [166]:
df = pd.read_csv('./NYCTaxiFares_small.csv')
df['pickup_datetime'].max()

'2010-04-25 03:59:42 UTC'

In [167]:
## Running this cell is really fucking slow, but it works.
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])

def havardsine_distance(lat1, long1, lat2, long2):
    r = 6371

    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)

    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(long2 - long1)
    a = np.sin(delta_phi / 2.0) ** 2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2.0) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    d = (r * c) # convert to kilometer
    return d

df['dist_km'] = df.apply(lambda row: havardsine_distance(row.pickup_latitude, row.pickup_longitude, row.dropoff_latitude, row.dropoff_longitude), axis=1)

airport_cords = [
    [40.691547, -74.180202], # NEWARK
    [40.773281, -73.869845], # LAGUARDIA
]

# When I wrote this code, only God and I knew what was going on. Today, only God knows.
def close_to_airport(values):
    pickup_latitude = values['pickup_latitude']
    pickup_longitude = values['pickup_longitude']
    dropoff_latitude = values['dropoff_latitude']
    dropoff_longitude = values['dropoff_longitude']

    # JFK is special, it needs a bigger radius
    jfkpickup = havardsine_distance(pickup_latitude, pickup_longitude, 40.645042, -73.786928) <= 1.00
    jfkdropoff = havardsine_distance(dropoff_latitude, dropoff_longitude, 40.645042, -73.786928) <= 1.00

    # Newark is a small airport but their parking log is BIG.
    newarkpickup = havardsine_distance(pickup_latitude, pickup_longitude, 40.691547, -74.180202) <= 1.00
    newarkdropoff = havardsine_distance(dropoff_latitude, dropoff_longitude, 40.691547, -74.180202) <= 1.00

    # Third largest, it's an abstract airport with a stretched out U shaped parking place
    # LaGuardia
    laguardiapickup = havardsine_distance(pickup_latitude, pickup_longitude, 40.773855, -73.871712) <= 0.50
    laguardiadropoff = havardsine_distance(dropoff_latitude, dropoff_longitude, 40.773855, -73.871712) <= 0.50

    return 1 if jfkpickup or jfkdropoff or newarkpickup or newarkdropoff or laguardiapickup or laguardiadropoff else 0

df['is_by_airport'] = df.apply(close_to_airport, axis=1)

In [168]:
my_time = df['pickup_datetime'][0]

In [169]:
df['EDTdate'] = df['pickup_datetime'] - pd.Timedelta(hours=4)
df['Hour'] = df['EDTdate'].dt.hour
df['AMorPM'] = np.where(df['Hour'] < 12, 0, 1)

In [170]:
df['Weekday'] = pd.to_numeric(df['EDTdate'].dt.strftime("%w")) # an int between 0 - 6, representing the weekdays
df['Month'] = pd.to_numeric(df['EDTdate'].dt.strftime("%m")) # an int between 0 - 6, representing the weekdays

#Formatting this correctly, we can use the dates to get the historical weather data for the dates
df['FullDate'] = df['EDTdate'].dt.strftime('%x')
#print(df['FullDate'].min())
#print(df['FullDate'].max())
print(df['FullDate'].unique())

['04/19/10' '04/17/10' '04/11/10' '04/16/10' '04/22/10' '04/23/10'
 '04/15/10' '04/20/10' '04/21/10' '04/14/10' '04/13/10' '04/12/10'
 '04/24/10' '04/18/10']


In [171]:
'''
api_key_openweather = "661b8c5b6e790b9cf55403d0af1cebe8"
new_york_middle_lat = ""
new_york_middle_long = ""
unique = np.sort(df['FullDate'].unique())
start = time.mktime(datetime.datetime.strptime(unique[0], "%m/%d/%y").timetuple())
end = time.mktime(datetime.datetime.strptime(unique[len(unique) - 1], "%m/%d/%y").timetuple())
url = f"https://history.openweathermap.org/data/2.5/history/city?lat={new_york_middle_lat}&lon={new_york_middle_long}&type=hour&start={start}&end={end}&appid={api_key_openweather}"

print(url)
'''
# Historical weather data, not working

'\napi_key_openweather = "661b8c5b6e790b9cf55403d0af1cebe8"\nnew_york_middle_lat = ""\nnew_york_middle_long = ""\nunique = np.sort(df[\'FullDate\'].unique())\nstart = time.mktime(datetime.datetime.strptime(unique[0], "%m/%d/%y").timetuple())\nend = time.mktime(datetime.datetime.strptime(unique[len(unique) - 1], "%m/%d/%y").timetuple())\nurl = f"https://history.openweathermap.org/data/2.5/history/city?lat={new_york_middle_lat}&lon={new_york_middle_long}&type=hour&start={start}&end={end}&appid={api_key_openweather}"\n\nprint(url)\n'

In [172]:
# lets have some weather-y fuuuuuuuuuuuun-fuuuuuun-function
# see this for doc https://www1.ncdc.noaa.gov/pub/data/cdo/documentation/GHCND_documentation.pdf
weather_data = pd.read_csv('nyc-weather-data.csv')
weather_labels = ['SNWD', 'SNOW', 'AWND'] #snow depth, did it snow
for index, item in enumerate(weather_labels):
    weather_data.loc[weather_data[item] <= -9999, item] = 0 # -9999 means that there is no data

# df['dist_km'] = df.apply(lambda row: havardsine_distance(row.pickup_latitude, row.pickup_longitude, row.dropoff_latitude, row.dropoff_longitude), axis=1)

weather_data['DATE'] = weather_data.apply(lambda row: datetime.datetime.strptime(str(row.DATE), '%Y%m%d').strftime('%m/%d/%y'), axis=1)
weather_data = weather_data[['DATE','PRCP','SNWD','SNOW','TMAX','TMIN','AWND']]

weather_data['TMIN'] = (weather_data['TMIN'] / 10) # the API returns degrees in an old format. We therefore convert the value to a tenth of its own form. Check the docu for this.
weather_data['TMAX'] = (weather_data['TMAX'] / 10)
weather_data['PRCP'] = weather_data['PRCP'] / 10

weather_data['AWND'] = weather_data['AWND'] / 2.237 # meters per second

df.rename(columns={'FullDate':'DATE'}, inplace=True)

merged_df = pd.merge(df, weather_data, how="left",on="DATE")

In [186]:
network_data = merged_df[['pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude', 'dist_km', 'passenger_count', 'is_by_airport', 'Hour', 'AMorPM', 'Weekday', 'Month', 'PRCP', 'SNWD', 'SNOW', 'TMAX', 'TMIN', 'AWND', 'fare_amount']]

prediction = network_data['fare_amount']

network_data.drop('fare_amount', axis=1)

X = network_data.values

y = prediction.values.reshape(-1,1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.FloatTensor(y_train)
y_test = torch.FloatTensor(y_test)

In [187]:
class NYTaxiFareModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(in_features=18, out_features=20)
        self.fc2 = nn.Linear(20, 20)
        self.out = nn.Linear(20, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.out(x)
        return x

model = NYTaxiFareModel()

In [179]:
criterion = nn.MSELoss()
optim = torch.optim.Adam(model.parameters(), lr=0.001)

In [188]:
epochs = 500
losses = []

for i in range(epochs):
    y_pred = model(X_train)
    loss = criterion(y_pred, y_train)
    losses.append(loss)

    if i % 25 == 1:
        print(f'epoch: {i:2} loss:{loss.item():10.8f}')
    optim.zero_grad()
    loss.backward()
    optim.step()

epoch:  1 loss:244.72637939
epoch: 26 loss:244.72637939
epoch: 51 loss:244.72637939
epoch: 76 loss:244.72637939
epoch: 101 loss:244.72637939
epoch: 126 loss:244.72637939
epoch: 151 loss:244.72637939
epoch: 176 loss:244.72637939
epoch: 201 loss:244.72637939
epoch: 226 loss:244.72637939
epoch: 251 loss:244.72637939
epoch: 276 loss:244.72637939
epoch: 301 loss:244.72637939
epoch: 326 loss:244.72637939
epoch: 351 loss:244.72637939
epoch: 376 loss:244.72637939
epoch: 401 loss:244.72637939
epoch: 426 loss:244.72637939
epoch: 451 loss:244.72637939
epoch: 476 loss:244.72637939
